# 理论介绍

HyDE 是一种创新方法，它将查询问题转换为包含答案的假设文档，旨在弥合向量空间中查询和文档分布之间的差距。传统的检索方法通常难以解决短查询和更长、更详细的文档之间的语义差距。HyDE 通过将查询扩展为完整的假设文档来解决这个问题，通过使查询表示更类似于向量空间中的文档表示，可能提高检索相关性。

![HyDE实现过程](figures/HyDE.jpeg)

实现步骤
1. PDF 处理和文本分块
2. 文档向量话：使用 Chroma 和 embedding 模型创建向量存储
3. 用于生成假设文档的语言模型：
    - 使用LL生成回答给定查询的假设文档。
    - 生成过程由提示模板引导，确保假设文档详细且与向量存储中使用的块大小相匹配。
4. 检索过程：基于 HyDE 技术实现自定义 HyDERetriever 类
    - 使用语言模型根据查询生成假设文档。
    - 使用假设文档作为向量存储中的搜索查询。
    - 检索与该假设文档最相似的文档块。

方法优点：
1. 提高相关性：通过将查询扩展到完整文档，HyDE 可以捕获更细致和相关的匹配
2. 处理复杂查询：对于可能难以直接匹配的复杂或多方面查询特别有用。
3. 提升跨领域属于理解：假设的文档生成可以适应不同类型的查询和文档领域。
4. 更好地理解上下文：扩展的查询可能会更好地捕获原始问题背后的上下文和意图。

# 代码实现

## 数据准备

In [42]:
import re
import json

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.chroma import Chroma

from langchain_openai import ChatOpenAI
from langchain.document_loaders.pdf import PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain import PromptTemplate

In [3]:
pdf_path = "data/pumpkin_book.pdf"
qa_path = 'data/train_dataset.json'
embedding = HuggingFaceEmbeddings(model_name='BAAI/bge-small-zh-v1.5')

with open(qa_path, 'r', encoding='utf-8') as file:
    qa_pairs = json.load(file)

In [43]:
def clean_text(text: str):
    """
    实现文本清理函数

    参数:
        text: 需要清理的字段

    返回:
        清理完成后返回的字段
    
    """
    # 删除每页开头与结尾标语及链接
    text = re.sub(r'→_→\n欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》\n←_←', '', text)
    text = re.sub(r'→_→\n配套视频教程：https://www.bilibili.com/video/BV1Mh411e7VU\n←_←', '', text)
    # 删除字符串开头的空格
    text = re.sub(r'\s+', '', text)
    # 删除回车
    text = re.sub(r'\n+', '', text)

    return text


def encode_pdf(path, chunk_size=1000, chunk_overlap=200):
    """
    使用 OpenAI 嵌入将 PDF 书籍编码为向量存储。

    参数:
        path: PDF 文件的路径。
        chunk_size: 每个文本块的期望大小。
        chunk_overlap: 连续块之间的重叠量。

    返回:
        包含内容的向量存储。
    """

    # 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径，加载PDF
    loader = PyMuPDFLoader(path)
    
    # 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
    pdf_pages = loader.load()
    
    # 第13页为南瓜书第一页正文，因此从13页开始,从倒数13页涉及敏感用语，因此从-13页结束
    data_pages = pdf_pages[13:-13]

    for page in data_pages:
        page.page_content = clean_text(page.page_content)

    # 文档分块
    text_splitter = CharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, separator='')

    split_docs = text_splitter.split_documents(data_pages)


    # 构建向量库
    vectordb = Chroma.from_documents(documents=split_docs, embedding=embedding)

    return vectordb

## 功能实现

In [48]:
class HyDERetriever:
    def __init__(self, chunk_size=400, chunk_overlap=50):
        self.llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=4000)
        self.embeddings = embedding
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.hyde_prompt = PromptTemplate(
            input_variables=["query", "chunk_size"],
            template="""给定问题"{query}"，生成一个直接回答该问题的假想文档。该文档应详细且深入，其长度必须正好为"{chunk_size}"个字符。"""
        )
        self.hyde_chain = self.hyde_prompt | self.llm

    # 基于pdf构建向量数据库    
    def encode_pdf_to_vectorstore(self, files_path):
        self.vectorstore = encode_pdf(files_path, chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap)

    # 生成假设问题
    def generate_hypothetical_document(self, query):
        input_variables = {"query": query, "chunk_size": self.chunk_size}
        return self.hyde_chain.invoke(input_variables).content

    # 就要假设问题召回top5
    def retrieve(self, query, k=5):
        hypothetical_doc = self.generate_hypothetical_document(query)
        similar_docs = self.vectorstore.similarity_search(hypothetical_doc, k=k)
        return similar_docs, hypothetical_doc

In [49]:
retriever = HyDERetriever()
retriever.encode_pdf_to_vectorstore(pdf_path)

In [52]:
test_doc = 1

qa_pairs[test_doc]

{'query': '请根据提供的上下文信息，解释什么是“泛化”能力，并给出一个具体的例子说明为何泛化能力是衡量机器学习模型好坏的关键。',
 'answer': '泛化：由于机器学习的目标是根据已知来对未知做出尽可能准确的判断，因此对未知事物判断的准确与否才是衡量一个模型好坏的关键，我们称此为“泛化”能力。例如学习西瓜好坏时，假设训练集中共有3个样本：{(x1=(青绿;蜷缩),y1=好瓜),(x2=(乌黑;蜷缩),y2=好瓜),(x3=(浅白;蜷缩),y3=好瓜)}，同时假设判断西瓜好坏的真相是“只要根蒂蜷缩就是好瓜”，如果应用算法A在此训练集上训练得到模型fa(x)，模型a学到的规律是“色泽等于青绿、乌黑或者浅白时，同时根蒂蜷缩即为好瓜，否则便是坏瓜”，再应用算法B在此训练集上训练得到模型fb(x)，模型fb(x)学到的规律是“只要根蒂蜷缩就是好瓜”，因此对于一个未见过的西瓜样本x=(金黄;蜷缩)来说，模型fa(x)给出的预测结果为“坏瓜”，模型fb(x)给出的预测结果为“好瓜”，此时我们称模型fb(x)的泛化能力优于模型fa(x)。通过以上举例可知，尽管模型fa(x)和模型fb(x)对训练集学得一样好，即两个模型对训练集中每个样本的判断都对，但是其所学到的规律是不同的。导致此现象最直接的原因是算法的不同，但是算法通常是有限的，可穷举的，尤其是在特定任务场景下可使用的算法更是有限，因此，数据便是导致此现象的另一重要原因，这也就是机器学习领域常说的“数据决定模型的上限，而算法则是让模型无限逼近上限”。',
 'page_num': 14}

In [53]:
test_query = qa_pairs[test_doc]['query']
results, hypothetical_doc = retriever.retrieve(test_query)

In [54]:
hypothetical_doc

'泛化能力是指机器学习模型在未见过的数据上表现良好的能力。它衡量模型是否能从训练数据中学到普遍规律，而非仅仅记住训练样本。泛化能力是模型好坏的关键，因为过拟合模型在训练集上表现优异，但在新数据上效果差。举例来说，垃圾邮件过滤器若具备良好泛化能力，能准确识别新型垃圾邮件，而非仅识别训练时见过的邮件。泛化能力确保模型在真实世界中具备实用性和可靠性。'

In [55]:
results

[Document(metadata={'author': '', 'creationDate': "D:20230303170709-00'00'", 'creator': 'LaTeX with hyperref', 'file_path': 'data/pumpkin_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 17, 'producer': 'xdvipdfmx (20200315)', 'source': 'data/pumpkin_book.pdf', 'subject': '', 'title': '', 'total_pages': 196, 'trapped': ''}, page_content='。泛化误差：学习器在新样本上的误差。经验误差和泛化误差用于分类问题的定义式可参见“西瓜书”第12章的式(12.1)和式(12.2)，接下来辨析一下以上几个概念。错误率和精度很容易理解，而且很明显是针对分类问题的。误差的概念更适用于回归问题，但是，根据“西瓜书”第12章的式(12.1)和式(12.2)的定义可以看出，在分类问题中也会使用误差的概念，此时的“差异”指的是学习器的实际预测输出的类别与样本真实的类别是否一致，若一致则“差异”为0，若不一致则“差异”为1，训练误差是在训练集上差异的平均值，而泛化误差则是在新样本（训练集中未出现过的样本）上差异的平均值。过拟合是由于模型的学习能力相对于数据来说过于强大，反过来说，欠拟合是因为模型的学习能力相对于数据来说过于低下。暂且抛开“没有免费的午餐”定理不谈，例如对于“西瓜书”第1章图1.4中的训练样本（黑点）来说，用类似于抛物线的曲线A去拟合则较为合理，而比较崎岖的曲线B相对于训练样本来说学习能力过于强大，但若仅用一条直线去训练则相对于训练样本来说直线的学习能力过于低下。2.2评估方法本节介绍了3种模型评估方法：留出法、交叉验证法、自助法。留'),
 Document(metadata={'author': '', 'creationDate': "D:20230303170709-00'00'", 'creator': 'LaTeX with hyperref', 'file_path': 'data/pump

## 效果测评

基于已经生成的QA数据集进行评测，由于HyDE需要生成问题，因此存在一定程度的token使用，我们这里使用前50个问题进行实验。

In [69]:
# 不实用HyDE召回率
from tqdm import tqdm

i = 0
j = 0
for qa_pair in tqdm(qa_pairs[:50]):
    if len(qa_pair['query']) > 10:
        query = qa_pair['query']
        sim_docs = retriever.vectorstore.similarity_search(query, k=5)
        page_nums = [doc.metadata['page'] for doc in sim_docs]
        if qa_pair['page_num'] in page_nums: i += 1
        j += 1

100%|██████████| 50/50 [00:06<00:00,  7.44it/s]


In [70]:
print(f"召回率为: {i/j * 100}%")

召回率为: 71.73913043478261%


In [63]:
from tqdm import tqdm

i = 0
j = 0
for qa_pair in tqdm(qa_pairs[:50]):
    if len(qa_pair['query']) > 10:
        query = qa_pair['query']
        sim_docs, hypothetical_doc = retriever.retrieve(query)
        page_nums = [doc.metadata['page'] for doc in sim_docs]
        if qa_pair['page_num'] in page_nums: i += 1
        j += 1

100%|██████████| 50/50 [02:40<00:00,  3.21s/it]


In [64]:
print(f"召回率为: {i/j * 100}%")

召回率为: 73.91304347826086%
